# Preprocessing

This notebook will give you a taste of what `scikit-learn` provides for preprocessing data.

## Data used
We will be using the planets data and red wine data:

### Data License for Planet Data
Copyright (C) 2012 Hanno Rein

Permission is hereby granted, free of charge, to any person obtaining a copy of this database and associated scripts (the "Database"), to deal in the Database without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Database, and to permit persons to whom the Database is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Database. A reference to the Database shall be included in all scientific publications that make use of the Database.

THE DATABASE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE DATABASE OR THE USE OR OTHER DEALINGS IN THE DATABASE.

### Citations for Red Wine Data
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
Modeling wine preferences by data mining from physicochemical properties.
In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.

Available at:
- [@Elsevier](http://dx.doi.org/10.1016/j.dss.2009.05.016)
- [Pre-press (pdf)](http://www3.dsi.uminho.pt/pcortez/winequality09.pdf)
- [bib](http://www3.dsi.uminho.pt/pcortez/dss09.bib)

Dua, D. and Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml/index.php](http://archive.ics.uci.edu/ml/index.php). Irvine, CA: University of California, School of Information and Computer Science.

## Setup

In [1]:
import numpy as np
import pandas as pd

planets = pd.read_csv('data/planets.csv')
red_wine = pd.read_csv('data/winequality-red.csv')
wine = pd.concat([
    pd.read_csv('data/winequality-white.csv', sep=';').assign(kind='white'), 
    red_wine.assign(kind='red')
])

## Train Test Split

Rather than having to write something like this every time:

```python
shuffled = planets.reindex(np.random.permutation(planets.index))
train_end_index = int(np.ceil(shuffled.shape[0] * .75))
training = shuffled.iloc[:train_end_index,]
testing = shuffled.iloc[train_end_index:,]
```

We can use scikit-learn's `train_test_split()` function to get our training and testing sets. (We will discuss the validation set in chapter 10.)

In [2]:
from sklearn.model_selection import train_test_split

X = planets[['eccentricity', 'semimajoraxis', 'mass']]
y = planets.period

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=0
)

The original data had this shape:

In [3]:
X.shape, y.shape

((5187, 3), (5187,))

Our training data has this shape:

In [4]:
X_train.shape, y_train.shape

((3890, 3), (3890,))

Our testing data has this shape:

In [5]:
X_test.shape, y_test.shape

((1297, 3), (1297,))

Let's look at the first 5 entries:

In [6]:
X_train.head()

,eccentricity,semimajoraxis,mass
4007,NaN,NaN,NaN
3976,NaN,NaN,NaN
2367,NaN,NaN,NaN
1031,0.04,NaN,0.02665
4371,NaN,1.52,0.05994


Our y data will be for the same rows as our X:

In [7]:
y_train.head()

4007    13.734698
3976    27.665523
2367     2.997932
1031    42.378000
4371          NaN
Name: period, dtype: float64

## Scaling data
### Standardizing with `StandardScaler`

In [8]:
from sklearn.preprocessing import StandardScaler

standardized = StandardScaler().fit_transform(X_train)

# examine some of the non-NaN values
standardized[~np.isnan(standardized)][:30]

array([-0.04097805, -0.28817547, -0.04000385, -0.28397187, -0.04497633,
       -0.04681797, -0.03732354, -0.046512  , -0.04818694, -0.25681579,
       -0.02575971, -0.04039483,  2.41320808, -0.046512  , -0.04818722,
       -0.08761115, -0.04707769, -0.28697209, -0.046512  , -0.02101369,
       -0.21438836, -0.04554356, -0.04702686, -0.28601874,  0.00938084,
       -0.04436042,  0.11000471, -0.00915787, -0.0478345 , -0.2864266 ])

### Normalizing with `MinMaxScaler`

In [9]:
from sklearn.preprocessing import MinMaxScaler

normalized = MinMaxScaler().fit_transform(X_train)

# examine some of the non-NaN values
normalized[~np.isnan(normalized)][:30]

array([6.04317391e-04, 1.01301303e-04, 2.34211256e-04, 2.27879253e-04,
       5.01150742e-04, 4.56776699e-05, 7.65490609e-03, 4.61526181e-04,
       7.80096345e-06, 1.04559791e-03, 9.96993220e-04, 2.23393755e-04,
       8.14448398e-02, 4.61526181e-04, 7.79323666e-06, 6.14065509e-03,
       3.84917586e-05, 1.37537045e-04, 4.61526181e-04, 7.59632724e-04,
       2.32316449e-03, 4.86514643e-04, 3.98980337e-05, 1.66244271e-04,
       1.90371741e-03, 1.13673389e-04, 1.20912256e-02, 1.42536685e-03,
       1.75521678e-05, 1.53962901e-04])

### Using the Median and IQR with `RobustScaler`

In [10]:
from sklearn.preprocessing import RobustScaler

robust_scaled = RobustScaler().fit_transform(X_train)

# examine some of the non-NaN values
robust_scaled[~np.isnan(robust_scaled)][:30]

array([-2.09026128e-01, -2.41560562e-01,  1.42606746e+00, -2.24571968e-01,
       -3.46318290e-01,  1.82818710e-01,  7.72242607e-01, -3.99049881e-01,
       -6.69520024e-02, -1.14822281e-01,  3.13539192e-01,  1.35473352e+00,
        1.06759205e+01, -3.99049881e-01, -6.70029553e-02,  5.69008191e-01,
        1.35432589e-01, -2.36697201e-01, -3.99049881e-01,  4.89085906e+00,
        5.66456584e-02, -3.65795724e-01,  1.44706002e-01, -2.32844275e-01,
        1.52019002e+00,  6.31203506e-01,  1.36766094e+00,  8.83610451e-01,
       -2.64954652e-03, -2.34492613e-01])

## Encoding
### Binary encoding with `np.where()`

In [11]:
np.where(wine.kind == 'red', 1, 0)

array([0, 0, 0, ..., 1, 1, 1])

We can also use the `LabelBinarizer` class from scikit-learn. By calling the `inverse_transform()` method, we see the labels assigned to each value:

In [12]:
from sklearn.preprocessing import LabelBinarizer

binary_labels = LabelBinarizer().fit(wine.kind)
binary_labels.inverse_transform(np.array([0, 1]))

array(['red', 'white'], dtype='<U5')

We can use the `Binarizer` class for binary encoding of values based on a threshold. Values less than or equal to `threshold` will be 0; values greater than `threshold` will be 1:

In [13]:
from sklearn.preprocessing import Binarizer

pd.Series(
    Binarizer(threshold=6).fit_transform(red_wine.quality.values.reshape(-1, 1)).flatten()
).value_counts()

0    1382
1     217
dtype: int64

### Ordinal Encoding with `LabelEncoder`

In [14]:
from sklearn.preprocessing import LabelEncoder

pd.Series(LabelEncoder().fit_transform(pd.cut(
    red_wine.quality,
    bins=[-1, 3, 6, 10],
    labels=['0-3 (low)', '4-6 (med)', '7-10 (high)']
))).value_counts()

1    1372
2     217
0      10
dtype: int64

### One-hot encoding
In some cases, label encoding may yield some associations that aren't something we want the model to be trained on. A safer strategy is to use one-hot encoding.

Our planets data has a `list` column that we can one-hot encode:

In [15]:
planets.list.value_counts()

Confirmed planets                    5060
Controversial                         100
Retracted planet candidate             12
Solar System                            9
Kepler Objects of Interest              5
Planets in binary systems, S-type       1
Name: list, dtype: int64

We can use `pd.get_dummies()` to one-hot encode this information:

In [16]:
pd.get_dummies(planets.list).head()

,Confirmed planets,Controversial,Kepler Objects of Interest,"Planets in binary systems, S-type",Retracted planet candidate,Solar System
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,0,1,0,0,0,0


This gives us a redundant column. Note that we only need one less column than the number of planet lists. Pandas makes it easy to remove one of the columns to address multicollinearity:

In [17]:
pd.get_dummies(planets.list, drop_first=True).head()

,Controversial,Kepler Objects of Interest,"Planets in binary systems, S-type",Retracted planet candidate,Solar System
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,1,0,0,0,0


We can also use the `LabelBinarizer` class:

In [18]:
from sklearn.preprocessing import LabelBinarizer

LabelBinarizer().fit_transform(planets.list)

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

## Imputing
The planets data has some missing values. We can use imputing strategies to avoid having to drop them from our model.

In [19]:
planets[['semimajoraxis', 'mass', 'eccentricity']].tail()

,semimajoraxis,mass,eccentricity
5182,NaN,0.42000,0.160
5183,114.00000,6.30000,NaN
5184,0.01557,0.00236,0.000
5185,0.02090,0.00308,0.040
5186,0.02764,0.00359,0.129


### `SimpleImputer`
We can fill with the `mean`, `median`, `most_frequent` (mode), or a constant value by specifiying the `strategy`. The default is the mean:

In [20]:
from sklearn.impute import SimpleImputer

SimpleImputer().fit_transform(
    planets[['semimajoraxis', 'mass', 'eccentricity']]
)

array([[1.290e+00, 1.940e+01, 2.310e-01],
       [1.540e+00, 1.120e+01, 8.000e-02],
       [8.300e-01, 4.800e+00, 0.000e+00],
       ...,
       [1.557e-02, 2.360e-03, 0.000e+00],
       [2.090e-02, 3.080e-03, 4.000e-02],
       [2.764e-02, 3.590e-03, 1.290e-01]])

Changing to the median is just a matter of passing that as the `strategy`:

In [21]:
from sklearn.impute import SimpleImputer

SimpleImputer(strategy='median').fit_transform(
    planets[['semimajoraxis', 'mass', 'eccentricity']]
)

array([[1.290e+00, 1.940e+01, 2.310e-01],
       [1.540e+00, 1.120e+01, 8.000e-02],
       [8.300e-01, 4.800e+00, 0.000e+00],
       ...,
       [1.557e-02, 2.360e-03, 0.000e+00],
       [2.090e-02, 3.080e-03, 4.000e-02],
       [2.764e-02, 3.590e-03, 1.290e-01]])

### `KNNImputer`
Since this data isn't something that is easily measured, assuming that the planets we don't have the data for are similar to the rest is dangerous. It could be that the ones that have missing data have something in common. Replacing missing values for the semi-major axis with the average of the ones we know is hardly a good strategy. Instead, we could try to use the mass and eccentricity columns to find similar planets and use their semi-major axes to impute the missing data. This can be done with the `KNNImputer` class. Notice the first column in the bottom 3 rows (the imputed semi-major axis is drastically different from what we got using the overall mean:

In [22]:
from sklearn.impute import KNNImputer

KNNImputer().fit_transform(
    planets[['semimajoraxis', 'mass', 'eccentricity']]
)

array([[1.290e+00, 1.940e+01, 2.310e-01],
       [1.540e+00, 1.120e+01, 8.000e-02],
       [8.300e-01, 4.800e+00, 0.000e+00],
       ...,
       [1.557e-02, 2.360e-03, 0.000e+00],
       [2.090e-02, 3.080e-03, 4.000e-02],
       [2.764e-02, 3.590e-03, 1.290e-01]])

### `MissingIndicator`
In some cases, we don't want to fill in a value, but rather use the fact that the data is missing as a feature in our model:

In [23]:
from sklearn.impute import MissingIndicator

MissingIndicator().fit_transform(
    planets[['semimajoraxis', 'mass', 'eccentricity']]
)

array([[False, False, False],
       [False, False, False],
       [False, False, False],
       ...,
       [False, False, False],
       [False, False, False],
       [False, False, False]])

## Additional Transformers
### `FunctionTransformer`
With the `FunctionTransformer` class, we can use any function on the data. By passing `validate=True`, we will convert the result to two-dimensional NumPy array and raise an error if there is an issue:

In [24]:
from sklearn.preprocessing import FunctionTransformer

FunctionTransformer(
    np.abs, validate=True
).fit_transform(X_train.dropna())

array([[1.110000e-02, 3.000000e-01, 2.013248e+00],
       [0.000000e+00, 5.490000e-02, 2.750000e-01],
       [1.500000e-01, 1.450000e+00, 2.142000e+01],
       ...,
       [1.200000e-01, 3.250000e+00, 1.450000e+00],
       [8.150000e-01, 1.530000e-01, 5.029000e+00],
       [8.370000e-03, 2.228038e-02, 9.340000e-04]])

### `ColumnTransformer`
Sometimes we don't want to perform the same transformation on all of our features, the `ColumnTransformer` class lets us specify which tranformations to use on each column. We pass a list of tuples in the form `(name, transformer object, columns to apply to)`:

In [25]:
from sklearn.compose import ColumnTransformer 
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

ColumnTransformer([
    ('impute', KNNImputer(), [0]),
    ('standard_scale', StandardScaler(), [1]),
    ('min_max', MinMaxScaler(), [2])
]).fit_transform(X_train)[10:15]

array([[ 0.00000000e+00, -4.81872173e-02,  6.14065509e-03],
       [ 3.36194182e-01,             nan,             nan],
       [ 3.36194182e-01, -4.70776889e-02,  1.37537045e-04],
       [ 3.36194182e-01,             nan,             nan],
       [ 0.00000000e+00, -2.10136861e-02,  2.32316449e-03]])

We can also use the `make_column_transformer()` function, which will name the transformers for us:

In [26]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical = [
    col for col in planets.columns
    if col in [
        'list', 'name', 'description', 
        'discoverymethod', 'lastupdate'
    ]
]
numeric = [col for col in planets.columns if col not in categorical]

make_column_transformer(
    (StandardScaler(), numeric),
    (OneHotEncoder(sparse=False), categorical)
).fit_transform(planets.dropna())

array([[-0.2476334 , -0.38337897, -0.01379043, ...,  0.        ,
         0.        ,  0.        ],
       [-0.25463945, -0.19829947, -0.68364849, ...,  0.        ,
         0.        ,  0.        ],
       [-0.25457404, -0.38337897, -1.03854018, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.61202438,  1.83757507, -1.03854018, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.37384427,  1.09725706, -0.87883892, ...,  0.        ,
         0.        ,  0.        ],
       [-0.22908264, -0.38337897, -1.03854018, ...,  0.        ,
         0.        ,  0.        ]])

## `Pipeline`
Using pipelines ensures the whole model training and testing process is consistent. To make a pipeline, we pass in a list of steps as tuples of `(name, object)`:

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

Pipeline([('scale', StandardScaler()), ('lr', LinearRegression())])

Pipeline(steps=[('scale', StandardScaler()), ('lr', LinearRegression())])

We aren't limited to using pipelines with models &mdash; they can be used inside other `sklearn` objects. This makes it possible for us to first use k-NN imputing on the semi-major axis data and then standard scale the result:

In [28]:
from sklearn.compose import ColumnTransformer 
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler

ColumnTransformer([
    ('impute', Pipeline([
        ('impute', KNNImputer()), ('scale', StandardScaler())
    ]), [0]),
    ('standard_scale', StandardScaler(), [1]),
    ('min_max', MinMaxScaler(), [2])
]).fit_transform(X_train)[10:15]

array([[-7.49271091e-02, -4.81872173e-02,  6.14065509e-03],
       [ 1.23716896e-17,             nan,             nan],
       [ 1.23716896e-17, -4.70776889e-02,  1.37537045e-04],
       [ 1.23716896e-17,             nan,             nan],
       [-7.49271091e-02, -2.10136861e-02,  2.32316449e-03]])

We can then include this as part of a pipeline, which gives us tremendous flexibility in how we build our models:

In [29]:
Pipeline([
    (
        'preprocessing', 
        ColumnTransformer([
            ('impute', Pipeline([
                ('impute', KNNImputer()), ('scale', StandardScaler())
            ]), [0]),
            ('standard_scale', StandardScaler(), [1]),
            ('min_max', MinMaxScaler(), [2])
        ])
    ),
    ('model', LinearRegression())
])

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('impute',
                                                  Pipeline(steps=[('impute',
                                                                   KNNImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  [0]),
                                                 ('standard_scale',
                                                  StandardScaler(), [1]),
                                                 ('min_max', MinMaxScaler(),
                                                  [2])])),
                ('model', LinearRegression())])

We can also use the `make_pipeline()` function to make the pipeline without naming the steps ourselves:

In [30]:
from sklearn.pipeline import make_pipeline

make_pipeline(StandardScaler(), LinearRegression())

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
        <a href="./planets_ml.ipynb">
            <button>Planets</button>
        </a>
        <a href="./red_wine.ipynb">
            <button>Red Wine</button>
        </a>
        <a href="./wine.ipynb">
            <button>Red + White Wine</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../ch_09/planets_ml.ipynb">
            <button>Next Notebook &#8594;</button>
        </a>
    </div>
</div>
<hr>